In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon as wilcoxon


# meta analysis functions
import sys
sys.path.append('/code/')
from util import analysis
from util import plotting
from util import util_continuous as utils

In [ ]:
# load the encoder estimation data

PATH = '/data/'

with open(PATH + 'encoder-estimation-data/encoder-decoder-data.pickle', 'rb') as handle:
    encoder, encoder_r2, idx_dict, pos_vel_model, pos, dec_vels, decoders = pickle.load(handle)
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']


with open(PATH + 'trial-related-data/times-decoder-update.pickle','rb') as handle:
    update_ix, update_times = pickle.load(handle)

In [ ]:
# key = keys[0]
# alphas = alphas_block1[key]
# print(alphas)

# conds = conditions_block2[key]
# print(conds)

# pDs = pDs_block1[key]
# print(pDs)

# # initialization conditions
# pos_init = [0, 1, 4, 5]
# neg_init = [2, 3, 6, 7]

# # penalty parameter conditions
# pD_3 = [0, 2, 4, 6] # pD = 1e-3
# pD_4 = [1, 3, 5, 7] # pD = 1e-4

# # learning rate conditions
# slow = [4, 5, 6, 7] 
# fast = [0, 1, 2, 3]

# # dimensions
# n_keys = len(keys) # number of subjects = length of keys
# n_blocks = 2 # number of blocks = 2
# # number of decoder conditions, time samples, number of emg channels 
# n_conds, n_time, n_feat = emgs_block1[keys[0]].shape 
# n_dim = 2 # number of dimensions of the task (2D trajectory tracking)
# x_dim = 8 # number of dimensions of encoder

# # set the time of the ramp at the start of each trial
# time_x = times_block1[keys[0]][0]
# ramp = np.where(time_x > 5)[0][0]
# print(ramp)

In [ ]:
# # trying to figure out how often decoder updates -- every 1202 samples

# W = Ws_block1[keys[0]][0]
# W[1:,:,:].shape # 7199 time points x (decoder dimensions is 2 x 6)
# dold = W[0]
# update_ix = []
# for ix,d in enumerate(W[1:]):
#   if (np.array_equal(dold,d)==False):
#     update_ix.append(ix)
#     dold = d

# update_ix.append(len(W) - 1) 
# update_ix = np.asarray(update_ix)
# update_ix = np.hstack([[0],update_ix])
# print("update index in time indices")
# print(update_ix)

# # only go up to 20432

# update_times = times_block1[keys[0]][0][update_ix]
# print("")
# print("update times in seconds")
# print(update_times)

# update_mins = update_times/60
# print("")
# print("update times in minutes")
# print(update_mins)

# tscale = update_ix[-1]/update_times[-1]
# print("")
# print("time scale conversion (index --> seconds): ", tscale)

# n_upd = len(update_ix) - 2 # last update is only 337

In [ ]:
idx_dict

In [ ]:
# check that the data is separated correctly

pos_err_slow = np.mean(np.linalg.norm(pos_vel_model[:, :, :, :, idx_dict['targ_pos_err_idx']]
                                      [:, :, utils.slow], axis = -1), axis = (0, 1, 2))
pos_err_fast = np.mean(np.linalg.norm(pos_vel_model[:, :, :, :, idx_dict['targ_pos_err_idx']]
                                      [:, :, utils.fast], axis = -1), axis = (0, 1, 2))
# know for sure that the error is more in the fast learning rate case
assert(np.mean(pos_err_fast)>np.mean(pos_err_slow))
plt.plot(pos_err_slow, label = 'slow', color = 'gray')
plt.plot(pos_err_fast, label = 'fast', color = 'green')
plt.legend()
plt.xlabel("time (s)")
plt.ylabel("time-domain error")
plt.title("Avg position error separated by learning rate ")
plt.show()

pos_err_pos = np.mean(np.linalg.norm(pos_vel_model[:, :, :, :, idx_dict['targ_pos_err_idx']][:, :, utils.pos_init], axis = -1), axis = (0, 1, 2))
pos_err_neg = np.mean(np.linalg.norm(pos_vel_model[:, :, :, :, idx_dict['targ_pos_err_idx']][:, :, utils.neg_init], axis = -1), axis = (0, 1, 2))


plt.plot(pos_err_pos, label = '+', color = 'orange')
plt.plot(pos_err_neg, label = '-', color = 'purple')
plt.legend()
plt.xlabel("time (s)")
plt.ylabel("time-domain error")
plt.title("Avg position error separated by initialization ")
plt.show()

pos_err_pd3 = np.mean(np.linalg.norm(pos_vel_model[:, :, :, :, idx_dict['targ_pos_err_idx']][:, :, utils.pD_3], axis = -1), axis = (0, 1, 2))
pos_err_pd4 = np.mean(np.linalg.norm(pos_vel_model[:, :, :, :, idx_dict['targ_pos_err_idx']][:, :, utils.pD_4], axis = -1), axis = (0, 1, 2))

plt.plot(pos_err_pd3, label = 'pd3 (high)')
plt.plot(pos_err_pd4, label = 'pd4 (low)')
plt.legend()
plt.title("Avg position error separated by penalty parameter ")
plt.xlabel("time (s)")
plt.ylabel("time-domain error")
plt.show()

In [ ]:
# # set up all decoders

# decoders = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(utils.update_ix)-2, 2, 64))

# for iK, key in enumerate(keys):
#     decoders[0, iK] = Ws_block1[key][:, update_ix[1:-1]]
#     decoders[1, iK] = Ws_block2[key][:, update_ix[1:-1]]

In [ ]:
decoders[:, :, utils.pos_init, 1:-1].shape

In [ ]:
# separate out conditions by slow and fast learning rates
slow_pos = list(set(utils.slow) & set(utils.pos_init))
slow_neg = list(set(utils.slow) & set(utils.neg_init))
fast_pos = list(set(utils.fast) & set(utils.pos_init))
fast_neg = list(set(utils.fast) & set(utils.neg_init))

In [ ]:
# take the mean per subject of all D+/E+, D-/E-, D+/E-, D-/E+, Dall, Eall

# all conditions
d_plus = np.mean(decoders[:, :, utils.pos_init, 1:-1], axis = (0, 2)) # (14, 17, 2, 64)
d_neg = np.mean(decoders[:, :, utils.neg_init, 1:-1], axis = (0, 2)) # want one average decoder over a trial per subject
d_all =  np.mean(decoders[:, :, :, 1:-1], axis = (0, 2))
assert(d_plus.shape == d_neg.shape == d_all.shape)
e_plus = np.mean(encoder[:, :, utils.pos_init, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
e_neg = np.mean(encoder[:, :, utils.neg_init, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
e_all = np.mean(encoder[:, :, :, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
assert(e_plus.shape == e_neg.shape == e_all.shape)


In [ ]:
d_neg.shape

In [ ]:
idx_dict

In [ ]:
# choosing encoder index
e_idx = idx_dict['targ_vel_err_idx']

if e_idx == [6, 7]:
    e_label = "B1"
elif e_idx == [2, 3]:
    e_label = "F1"

In [ ]:
# D+/E+


dp_ep = np.median(d_plus@e_plus[:, :, :, e_idx],axis = 0) # take median across subjects
dn_en = np.median(d_neg@e_neg[:, :, :, e_idx],axis = 0) # take median across subjects
dp_en = np.median(d_plus@e_neg[:, :, :, e_idx],axis = 0) # take median across subjects
dn_ep = np.median(d_neg@e_plus[:, :, :, e_idx],axis = 0) # take median across subjects
dall_eall = np.median(d_all@e_all[:, :, :, e_idx],axis = 0) # take median across subjects


fig,ax = plt.subplots(2, 2,sharex=True, figsize=(10, 6))

## D+ E+
ax[0, 0].plot(dp_ep[:, 0, 0], color = 'red')
ax[0, 1].plot(dp_ep[:, 0, 1], color = 'red', label = "D+E+")
ax[1, 0].plot(dp_ep[:, 1, 0], color = 'red')
ax[1, 1].plot(dp_ep[:, 1, 1], color = 'red')

## D- E-
ax[0, 0].plot(dn_en[:, 0, 0], color = 'orange')
ax[0, 1].plot(dn_en[:, 0, 1], color = 'orange', label = "D-E-")
ax[1, 0].plot(dn_en[:, 1, 0], color = 'orange')
ax[1, 1].plot(dn_en[:, 1, 1], color = 'orange')

## D+ E-
ax[0, 0].plot(dp_en[:, 0, 0], color = 'green')
ax[0, 1].plot(dp_en[:, 0, 1], color = 'green', label = "D+E-")
ax[1, 0].plot(dp_en[:, 1, 0], color = 'green')
ax[1, 1].plot(dp_en[:, 1, 1], color = 'green')

## D- E+
ax[0, 0].plot(dn_ep[:, 0, 0], color = 'blue')
ax[0, 1].plot(dn_ep[:, 0, 1], color = 'blue', label = "D-E+")
ax[1, 0].plot(dn_ep[:, 1, 0], color = 'blue')
ax[1, 1].plot(dn_ep[:, 1, 1], color = 'blue')

## D-all E-all
ax[0, 0].plot(dall_eall[:, 0, 0], color = 'purple')
ax[0, 1].plot(dall_eall[:, 0, 1], color = 'purple', label = "DE")
ax[1, 0].plot(dall_eall[:, 1, 0], color = 'purple')
ax[1, 1].plot(dall_eall[:, 1, 1], color = 'purple')

ax[0, 0].set_title("(0, 0)")
ax[1, 0].set_title("(1, 0)")
ax[0, 1].set_title("(0, 1)")
ax[1, 1].set_title("(1, 1)")

ax[0, 0].axhline(y=1,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[0, 1].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[1, 0].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[1, 1].axhline(y=1,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)

# ax[0, 1].legend()
# Hide the handles for legend
handles, labels = ax[0, 1].get_legend_handles_labels()
ax[0, 1].legend([], [], frameon=False)

# Create a legend outside ax[0, 1] with text color matching line color
ax[0, 1].legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5), handlelength=0, handletextpad=0,
             fontsize='large', title='Legend', title_fontsize='large', labelcolor = ['red', 'orange', 'green', 'blue', 'purple'])

fig.suptitle("DE Pairs | Slow + Fast | " + e_label)

In [ ]:
dp_ep_last = np.mean((d_plus@e_plus)[:, -3:, :, e_idx], axis = 1)
dn_en_last = np.mean((d_neg@e_neg)[:, -3:, :, e_idx], axis = 1)
dp_match = np.vstack((dp_ep_last, dn_en_last))
# dp_match_mean = np.mean(dp_match, axis = 0)
dp_match_mean = np.mean(np.stack((dp_ep_last, dn_en_last), axis = 3), axis = 3)


dp_en_last = np.mean((d_plus@e_neg)[:, -3:, :, e_idx], axis = 1)
dn_ep_last = np.mean((d_neg@e_plus)[:, -3:, :, e_idx], axis = 1)
dp_mismatch_mean = np.mean(np.stack((dp_en_last, dn_ep_last), axis = 3), axis = 3)
dp_mismatch = np.vstack((dp_en_last, dn_ep_last))
# dp_mismatch_mean = np.mean(dp_mismatch, axis = 0)

dall_eall_last = np.mean((d_all@e_all)[:, -3:, :, e_idx], axis = 1)


# wilcoxon of dpep vs dnen
w_pp1 = wilcoxon(dp_ep_last[:, 0, 0],  dn_en_last[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_ep_last[:, 0, 1],  dn_en_last[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_ep_last[:, 1, 0],  dn_en_last[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_ep_last[:, 1, 1],  dn_en_last[:, 1, 1])
print(w_pp4)
print("")


# wilcoxon of dpen vs dnep
w_pp1 = wilcoxon(dp_en_last[:, 0, 0],  dn_ep_last[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_en_last[:, 0, 1],  dn_ep_last[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_en_last[:, 1, 0],  dn_ep_last[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_en_last[:, 1, 1],  dn_ep_last[:, 1, 1])
print(w_pp4)
print("")


# wilcoxon of match vs mismatch - N = 14 * 2
print(dp_match[:, 0, 0].shape)
w_pp1 = wilcoxon(dp_match[:, 0, 0],  dp_mismatch[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_match[:, 0, 1],  dp_mismatch[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_match[:, 1, 0],  dp_mismatch[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_match[:, 1, 1],  dp_mismatch[:, 1, 1])
print(w_pp4)
print("")



## slow only

In [ ]:
# use only the slow conditions
d_plus = np.mean(decoders[:, :, slow_pos, 1:-1], axis = (0, 2)) # (14, 17, 2, 64)
d_neg = np.mean(decoders[:, :, slow_neg, 1:-1], axis = (0, 2)) # want one average decoder over a trial per subject
d_all =  np.mean(decoders[:, :, utils.slow, 1:-1], axis = (0, 2))
assert(d_plus.shape == d_neg.shape == d_all.shape)
e_plus = np.mean(encoder[:, :, slow_pos, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
e_neg = np.mean(encoder[:, :, slow_neg, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
e_all = np.mean(encoder[:, :, utils.slow, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
assert(e_plus.shape == e_neg.shape == e_all.shape)

In [ ]:
# D+/E+

dp_ep_slow = d_plus@e_plus[:, :, :, e_idx]
dn_en_slow = d_neg@e_neg[:, :, :, e_idx]
dp_en_slow = d_plus@e_neg[:, :, :, e_idx]
dn_ep_slow = d_neg@e_plus[:, :, :, e_idx]


dp_ep = np.median(dp_ep_slow,axis = 0) # take median across subjects
dn_en = np.median(dn_en_slow,axis = 0) # take median across subjects
dp_en = np.median(dp_en_slow,axis = 0) # take median across subjects
dn_ep = np.median(dn_ep_slow,axis = 0) # take median across subjects
dall_eall = np.median(d_all@e_all[:, :, :, e_idx],axis = 0) # take median across subjects


fig,ax = plt.subplots(2, 2,sharex=True, figsize=(10, 6))

## D+ E+
ax[0, 0].plot(dp_ep[:, 0, 0], color = 'red')
ax[0, 1].plot(dp_ep[:, 0, 1], color = 'red', label = "D+E+")
ax[1, 0].plot(dp_ep[:, 1, 0], color = 'red')
ax[1, 1].plot(dp_ep[:, 1, 1], color = 'red')

## D- E-
ax[0, 0].plot(dn_en[:, 0, 0], color = 'orange')
ax[0, 1].plot(dn_en[:, 0, 1], color = 'orange', label = "D-E-")
ax[1, 0].plot(dn_en[:, 1, 0], color = 'orange')
ax[1, 1].plot(dn_en[:, 1, 1], color = 'orange')

## D+ E-
ax[0, 0].plot(dp_en[:, 0, 0], color = 'green')
ax[0, 1].plot(dp_en[:, 0, 1], color = 'green', label = "D+E-")
ax[1, 0].plot(dp_en[:, 1, 0], color = 'green')
ax[1, 1].plot(dp_en[:, 1, 1], color = 'green')

## D- E+
ax[0, 0].plot(dn_ep[:, 0, 0], color = 'blue')
ax[0, 1].plot(dn_ep[:, 0, 1], color = 'blue', label = "D-E+")
ax[1, 0].plot(dn_ep[:, 1, 0], color = 'blue')
ax[1, 1].plot(dn_ep[:, 1, 1], color = 'blue')

## D-all E-all
ax[0, 0].plot(dall_eall[:, 0, 0], color = 'purple')
ax[0, 1].plot(dall_eall[:, 0, 1], color = 'purple', label = "DE")
ax[1, 0].plot(dall_eall[:, 1, 0], color = 'purple')
ax[1, 1].plot(dall_eall[:, 1, 1], color = 'purple')

ax[0, 0].set_title("(0, 0)")
ax[1, 0].set_title("(1, 0)")
ax[0, 1].set_title("(0, 1)")
ax[1, 1].set_title("(1, 1)")

ax[0, 0].axhline(y=1,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[0, 1].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[1, 0].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[1, 1].axhline(y=1,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)

# ax[0, 1].legend()
# Hide the handles for legend
handles, labels = ax[0, 1].get_legend_handles_labels()
ax[0, 1].legend([], [], frameon=False)

# Create a legend outside ax[0, 1] with text color matching line color
ax[0, 1].legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5), handlelength=0, handletextpad=0,
             fontsize='large', title='Legend', title_fontsize='large', labelcolor = ['red', 'orange', 'green', 'blue', 'purple'])

fig.suptitle("DE Pairs | Slow | " + e_label)

In [ ]:
# find the last minute values -- will be 14 x 2 x 2 
dp_ep_last = np.mean(dp_ep_slow[:, -3:], axis = 1)
dn_en_last = np.mean(dn_en_slow[:, -3:], axis = 1)
dp_match_slow = np.vstack((dp_ep_last, dn_en_last))
# dp_match_mean = np.mean(dp_match, axis = 0)
dp_match_mean = np.mean(np.stack((dp_ep_last, dn_en_last), axis = 3), axis = 3)


dp_en_last = np.mean(dp_en_slow[:, -3:], axis = 1)
dn_ep_last = np.mean(dn_ep_slow[:, -3:], axis = 1)
dp_mismatch_mean = np.mean(np.stack((dp_en_last, dn_ep_last), axis = 3), axis = 3)
dp_mismatch_slow = np.vstack((dp_en_last, dn_ep_last))
# dp_mismatch_mean = np.mean(dp_mismatch, axis = 0)

dall_eall_last = np.mean((d_all@e_all)[:, -3:, :, e_idx], axis = 1)


# wilcoxon of dpep vs dnen
print("wilcoxon of dpep vs dnen")
w_pp1 = wilcoxon(dp_ep_last[:, 0, 0],  dn_en_last[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_ep_last[:, 0, 1],  dn_en_last[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_ep_last[:, 1, 0],  dn_en_last[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_ep_last[:, 1, 1],  dn_en_last[:, 1, 1])
print(w_pp4)
print("")


# wilcoxon of dpen vs dnep
print("wilcoxon of dpen vs dnep")
w_pp1 = wilcoxon(dp_en_last[:, 0, 0],  dn_ep_last[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_en_last[:, 0, 1],  dn_ep_last[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_en_last[:, 1, 0],  dn_ep_last[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_en_last[:, 1, 1],  dn_ep_last[:, 1, 1])
print(w_pp4)
print("")


# wilcoxon of match vs mismatch - N = 14 * 2
print("wilcoxon of match vs mismatch")
print(dp_match[:, 0, 0].shape)
w_pp1 = wilcoxon(dp_match_slow[:, 0, 0],  dp_mismatch_slow[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_match_slow[:, 0, 1],  dp_mismatch_slow[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_match_slow[:, 1, 0],  dp_mismatch_slow[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_match_slow[:, 1, 1],  dp_mismatch_slow[:, 1, 1])
print(w_pp4)
print("")



## Fast only

In [ ]:
# use only the slow conditions
d_plus = np.mean(decoders[:, :, fast_pos, 1:-1], axis = (0, 2)) # (14, 17, 2, 64)
d_neg = np.mean(decoders[:, :, fast_neg, 1:-1], axis = (0, 2)) # want one average decoder over a trial per subject
d_all =  np.mean(decoders[:, :, utils.fast, 1:-1], axis = (0, 2))
assert(d_plus.shape == d_neg.shape == d_all.shape)
e_plus = np.mean(encoder[:, :, fast_pos, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
e_neg = np.mean(encoder[:, :, fast_neg, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
e_all = np.mean(encoder[:, :, utils.fast, :, :, :-1], axis = (0, 2)) # (14, 17, 64, 8)
assert(e_plus.shape == e_neg.shape == e_all.shape)

In [ ]:
# D+/E+


dp_ep_fast = d_plus@e_plus[:, :, :, e_idx]
dn_en_fast = d_neg@e_neg[:, :, :, e_idx]
dp_en_fast = d_plus@e_neg[:, :, :, e_idx]
dn_ep_fast = d_neg@e_plus[:, :, :, e_idx]


dp_ep = np.median(dp_ep_fast,axis = 0) # take median across subjects
dn_en = np.median(dn_en_fast,axis = 0) # take median across subjects
dp_en = np.median(dp_en_fast,axis = 0) # take median across subjects
dn_ep = np.median(dn_ep_fast,axis = 0) # take median across subjects

dall_eall = np.median(d_all@e_all[:, :, :, e_idx],axis = 0) # take median across subjects


fig,ax = plt.subplots(2, 2,sharex=True, figsize=(10, 6))

## D+ E+
ax[0, 0].plot(dp_ep[:, 0, 0], color = 'red')
ax[0, 1].plot(dp_ep[:, 0, 1], color = 'red', label = "D+E+")
ax[1, 0].plot(dp_ep[:, 1, 0], color = 'red')
ax[1, 1].plot(dp_ep[:, 1, 1], color = 'red')

## D- E-
ax[0, 0].plot(dn_en[:, 0, 0], color = 'orange')
ax[0, 1].plot(dn_en[:, 0, 1], color = 'orange', label = "D-E-")
ax[1, 0].plot(dn_en[:, 1, 0], color = 'orange')
ax[1, 1].plot(dn_en[:, 1, 1], color = 'orange')

## D+ E-
ax[0, 0].plot(dp_en[:, 0, 0], color = 'green')
ax[0, 1].plot(dp_en[:, 0, 1], color = 'green', label = "D+E-")
ax[1, 0].plot(dp_en[:, 1, 0], color = 'green')
ax[1, 1].plot(dp_en[:, 1, 1], color = 'green')

## D- E+
ax[0, 0].plot(dn_ep[:, 0, 0], color = 'blue')
ax[0, 1].plot(dn_ep[:, 0, 1], color = 'blue', label = "D-E+")
ax[1, 0].plot(dn_ep[:, 1, 0], color = 'blue')
ax[1, 1].plot(dn_ep[:, 1, 1], color = 'blue')

## D-all E-all
ax[0, 0].plot(dall_eall[:, 0, 0], color = 'purple')
ax[0, 1].plot(dall_eall[:, 0, 1], color = 'purple', label = "DE")
ax[1, 0].plot(dall_eall[:, 1, 0], color = 'purple')
ax[1, 1].plot(dall_eall[:, 1, 1], color = 'purple')

ax[0, 0].set_title("(0, 0)")
ax[1, 0].set_title("(1, 0)")
ax[0, 1].set_title("(0, 1)")
ax[1, 1].set_title("(1, 1)")

ax[0, 0].axhline(y=1,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[0, 1].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[1, 0].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)
ax[1, 1].axhline(y=1,xmin=0,xmax=5,c="black",ls = '--', linewidth=3,zorder=0)

# ax[0, 1].legend()
# Hide the handles for legend
handles, labels = ax[0, 1].get_legend_handles_labels()
ax[0, 1].legend([], [], frameon=False)

# Create a legend outside ax[0, 1] with text color matching line color
ax[0, 1].legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5), handlelength=0, handletextpad=0,
             fontsize='large', title='Legend', title_fontsize='large', labelcolor = ['red', 'orange', 'green', 'blue', 'purple'])

fig.suptitle("DE Pairs | Fast only | " + e_label) 

In [ ]:
# find the last minute values -- will be 14 x 2 x 2 
dp_ep_last = np.mean(dp_ep_fast[:, -3:], axis = 1)
dn_en_last = np.mean(dn_en_fast[:, -3:], axis = 1)
dp_match_fast = np.vstack((dp_ep_last, dn_en_last))
# dp_match_mean = np.mean(dp_match, axis = 0)
dp_match_mean = np.mean(np.stack((dp_ep_last, dn_en_last), axis = 3), axis = 3)


dp_en_last = np.mean(dp_en_fast[:, -3:], axis = 1)
dn_ep_last = np.mean(dn_ep_fast[:, -3:], axis = 1)
dp_mismatch_mean = np.mean(np.stack((dp_en_last, dn_ep_last), axis = 3), axis = 3)
dp_mismatch_fast = np.vstack((dp_en_last, dn_ep_last))
# dp_mismatch_mean = np.mean(dp_mismatch, axis = 0)

dall_eall_last = np.mean((d_all@e_all)[:, -3:, :, e_idx], axis = 1)

# wilcoxon of dpep vs dnen
w_pp1 = wilcoxon(dp_ep_last[:, 0, 0],  dn_en_last[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_ep_last[:, 0, 1],  dn_en_last[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_ep_last[:, 1, 0],  dn_en_last[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_ep_last[:, 1, 1],  dn_en_last[:, 1, 1])
print(w_pp4)
print("")


# wilcoxon of dpen vs dnep
w_pp1 = wilcoxon(dp_en_last[:, 0, 0],  dn_ep_last[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_en_last[:, 0, 1],  dn_ep_last[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_en_last[:, 1, 0],  dn_ep_last[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_en_last[:, 1, 1],  dn_ep_last[:, 1, 1])
print(w_pp4)
print("")


# wilcoxon of match vs mismatch - N = 14 * 2
print(dp_match[:, 0, 0].shape)
w_pp1 = wilcoxon(dp_match_fast[:, 0, 0],  dp_mismatch_fast[:, 0, 0])
print(w_pp1)
w_pp2 = wilcoxon(dp_match_fast[:, 0, 1],  dp_mismatch_fast[:, 0, 1])
print(w_pp2)
w_pp3 = wilcoxon(dp_match_fast[:, 1, 0],  dp_mismatch_fast[:, 1, 0])
print(w_pp3)
w_pp4 = wilcoxon(dp_match_fast[:, 1, 1],  dp_mismatch_fast[:, 1, 1])
print(w_pp4)
print("")

# # wilcoxon of mismatch vs all 
# w_pp1 = wilcoxon(dall_eall_last[:, 0, 0],  dp_match_mean[:, 0, 0])
# print(w_pp1)
# w_pp2 = wilcoxon(dall_eall_last[:, 0, 1],  dp_match_mean[:, 0, 1])
# print(w_pp2)
# w_pp3 = wilcoxon(dall_eall_last[:, 1, 0],  dp_match_mean[:, 1, 0])
# print(w_pp3)
# w_pp4 = wilcoxon(dall_eall_last[:, 1, 1],  dp_match_mean[:, 1, 1])
# print(w_pp4)
# print("")

In [ ]:
import matplotlib.ticker as ticker

label_size = 6
## SETUP THE FIGURE HERE
## HAVE TO RE-REUN FROM HERE TO "CLEAR" THE PLOT
fig_decinit = plt.figure(figsize = (3, 3), layout='constrained') # set the total figure size
mosaic = """
    ab
    cd
    """

# set up the axes
ax_dict = fig_decinit.subplot_mosaic(mosaic)
for ii in ax_dict:
    plotting.remove_and_set_axes(ax_dict[ii], bottom=True, left=True)
    ax_dict[ii].tick_params(axis='both', which='major', labelsize = label_size)
    ax_dict[ii].tick_params(axis='both', which='minor', labelsize = label_size)
    # set the x-ticks at multiples of 1
    ax_dict[ii].xaxis.set_major_locator(ticker.MultipleLocator(base=5))
    ax_dict[ii].xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
fig_decinit.patch.set_facecolor('white')


fig_decinit.suptitle("$D \cdot $" + e_label, fontsize = 7)
ax_dict['a'].set_ylabel("(0, 0)")
ax_dict['b'].set_ylabel("(0, 1)")
ax_dict['c'].set_ylabel("(1, 0)")
ax_dict['d'].set_ylabel("(1, 1)")

In [ ]:
## add to boxplot

## add subfigure to the combined plots

# a - 0,0
axs = ax_dict['a']
data_groups = [dp_match_slow[:, 0, 0], dp_mismatch_slow[:, 0, 0], dp_match_fast[:, 0, 0], dp_mismatch_fast[:, 0, 0]]
data_labels = ['match (slow)', 'mismatch (slow)', 'match (fast)', 'mismatch (fast)']
data_pos = [0, 0.4, 1, 1.4]
bplot = axs.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths=0.2,
                    boxprops=dict(edgecolor="none"),
                    medianprops=dict(color='k', lw=1))


for patch, color in zip(bplot['boxes'], 
                               ['blue', 'gray', 'orange', 'gray']):
            patch.set_facecolor(color)
        
# rotate labels  
axs.set_xticks(data_pos,data_labels, rotation=40)

print("wilcoxon of match vs mismatch")
print(dp_match[:, 0, 0].shape)

w00_slow = wilcoxon(dp_match_slow[:, 0, 0],  dp_mismatch_slow[:, 0, 0])
print(w00_slow)

plotting.plot_significance(pvalue = w00_slow.pvalue, data1=dp_match_slow[:, 0, 0], data2 = dp_mismatch_slow[:, 0, 0], 
                           data_pos = data_pos[0:2], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1, y_asterix=0.4, y_bar=0.2)

w00_fast = wilcoxon(dp_match_fast[:, 0, 0],  dp_mismatch_fast[:, 0, 0])
print(w00_fast)

plotting.plot_significance(pvalue = w00_fast.pvalue, data1=dp_match_fast[:, 0, 0], data2 = dp_mismatch_fast[:, 0, 0], 
                           data_pos = data_pos[2:4], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1,y_asterix=0.4, y_bar=0.2)

### b - 0,1
axs = ax_dict['b']
data_groups = [dp_match_slow[:, 0, 1], dp_mismatch_slow[:, 0, 1], dp_match_fast[:, 0, 1], dp_mismatch_fast[:, 0, 1]]
data_labels = ['match (slow)', 'mismatch (slow)', 'match (fast)', 'mismatch (fast)']
data_pos = [0, 0.4, 1, 1.4]
bplot = axs.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths=0.2,
                    boxprops=dict(edgecolor="none"),
                    medianprops=dict(color='k', lw=1))


for patch, color in zip(bplot['boxes'], 
                               ['blue', 'gray', 'orange', 'gray']):
            patch.set_facecolor(color)
        
# rotate labels  
axs.set_xticks(data_pos,data_labels, rotation=40)

print("wilcoxon of match vs mismatch")
print(dp_match[:, 0, 0].shape)

w01_slow = wilcoxon(dp_match_slow[:, 0, 1],  dp_mismatch_slow[:, 0, 1])
print(w01_slow)

plotting.plot_significance(pvalue = w01_slow.pvalue, data1=dp_match_slow[:, 0, 1], data2 = dp_mismatch_slow[:, 0, 1], 
                           data_pos = data_pos[0:2], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1, y_asterix=0.4, y_bar=0.2)

w01_fast = wilcoxon(dp_match_fast[:, 0, 1],  dp_mismatch_fast[:, 0,1])
print(w01_fast)

plotting.plot_significance(pvalue = w01_fast.pvalue, data1=dp_match_fast[:, 0, 1], data2 = dp_mismatch_fast[:, 0, 1], 
                           data_pos = data_pos[2:4], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1,y_asterix=0.4, y_bar=0.2)


### c - 1, 0
axs = ax_dict['c']
data_groups = [dp_match_slow[:, 1, 0], dp_mismatch_slow[:, 1, 0], dp_match_fast[:, 1, 0], dp_mismatch_fast[:, 1, 0]]
data_labels = ['match (slow)', 'mismatch (slow)', 'match (fast)', 'mismatch (fast)']
data_pos = [0, 0.4, 1, 1.4]
bplot = axs.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths=0.2,
                    boxprops=dict(edgecolor="none"),
                    medianprops=dict(color='k', lw=1))


for patch, color in zip(bplot['boxes'], 
                               ['blue', 'gray', 'orange', 'gray']):
            patch.set_facecolor(color)
        
# rotate labels  
axs.set_xticks(data_pos,data_labels, rotation=40)

print("wilcoxon of match vs mismatch")
print(dp_match[:, 1, 0].shape)

w10_slow = wilcoxon(dp_match_slow[:, 1, 0],  dp_mismatch_slow[:, 1, 0])
print(w10_slow)

plotting.plot_significance(pvalue = w10_slow.pvalue, data1=dp_match_slow[:, 1, 0], data2 = dp_mismatch_slow[:, 1, 0], 
                           data_pos = data_pos[0:2], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1, y_asterix=0.4, y_bar=0.2)

w10_fast = wilcoxon(dp_match_fast[:, 1, 0],  dp_mismatch_fast[:, 1, 0])
print(w10_fast)

plotting.plot_significance(pvalue = w10_fast.pvalue, data1=dp_match_fast[:, 1, 0], data2 = dp_mismatch_fast[:, 1, 0], 
                           data_pos = data_pos[2:4], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1,y_asterix=0.4, y_bar=0.2)


### d -- 1, 1
axs = ax_dict['d']
data_groups = [dp_match_slow[:, 1, 1], dp_mismatch_slow[:, 1, 1], dp_match_fast[:, 1, 1], dp_mismatch_fast[:, 1, 1]]
data_labels = ['match (slow)', 'mismatch (slow)', 'match (fast)', 'mismatch (fast)']
data_pos = [0, 0.4, 1, 1.4]
bplot = axs.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths=0.2,
                    boxprops=dict(edgecolor="none"),
                    medianprops=dict(color='k', lw=1))


for patch, color in zip(bplot['boxes'], 
                               ['blue', 'gray', 'orange', 'gray']):
            patch.set_facecolor(color)
        
# rotate labels  
axs.set_xticks(data_pos,data_labels, rotation=40)

print("wilcoxon of match vs mismatch")
print(dp_match[:, 1, 1].shape)

w11_slow = wilcoxon(dp_match_slow[:, 1, 1],  dp_mismatch_slow[:, 1, 1])
print(w11_slow)

plotting.plot_significance(pvalue = w11_slow.pvalue, data1=dp_match_slow[:, 1, 1], data2 = dp_mismatch_slow[:, 1, 1], 
                           data_pos = data_pos[0:2], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1, y_asterix=0.4, y_bar=0.2)

w11_fast = wilcoxon(dp_match_fast[:, 1, 1],  dp_mismatch_fast[:, 1, 1])
print(w11_fast)

plotting.plot_significance(pvalue = w11_fast.pvalue, data1=dp_match_fast[:, 1, 1], data2 = dp_mismatch_fast[:, 1, 1], 
                           data_pos = data_pos[2:4], fig=fig_decinit, 
                           ax=axs, fontsize=10, lw=1,y_asterix=0.4, y_bar=0.2)

fig_decinit

In [ ]:
image_format = 'pdf' # e.g .png, .svg, etc.
image_name = 'sfig-init-' + e_label + ".pdf"
PATH = '/results/'
fig_decinit.savefig(PATH + image_name, format=image_format, dpi=300)